In [1]:
from simple.binance import getHistMonth
from simple.plotly import interactFigure
from simple.pretty import pmap
import pandas as pd
from calendar import day_name

In [2]:
frame = '1h'
tickers = ['XLMUSDT', 'ETHUSDT', 'LTCUSDT', 'TRXUSDT', 'BNBUSDT', 'ADAUSDT',
           'BTCUSDT', 'ETCUSDT', 'XRPUSDT', 'XMRUSDT', 'ZECUSDT', 'EOSUSDT']
weekdays = [f'{day: >20}' for day in day_name]

In [3]:
def getDiff(ticker):
    return getHistMonth('2019-01', '2022-05', ticker, frame, close_only=True, spot=True).diff().dropna()

In [4]:
F = pd.concat(pmap(getDiff, tickers), axis=1)
F = F.join(F.index.isocalendar())
F['week_hour'] = F.groupby(['year', 'week']).cumcount()
F

  0%|          | 0/12 [00:00<?, ?it/s]

XMRUSDT https://data.binance.vision/data/spot/monthly/klines/XMRUSDT/1h/XMRUSDT-1h-2019-02.zip HTTP Error 404: Not Found
ZECUSDT https://data.binance.vision/data/spot/monthly/klines/ZECUSDT/1h/ZECUSDT-1h-2019-01.zip HTTP Error 404: Not Found
ZECUSDT https://data.binance.vision/data/spot/monthly/klines/ZECUSDT/1h/ZECUSDT-1h-2019-02.zip HTTP Error 404: Not Found
XMRUSDT https://data.binance.vision/data/spot/monthly/klines/XMRUSDT/1h/XMRUSDT-1h-2019-01.zip HTTP Error 404: Not Found


,XLMUSDT,ETHUSDT,LTCUSDT,TRXUSDT,BNBUSDT,ADAUSDT,BTCUSDT,ETCUSDT,XRPUSDT,XMRUSDT,ZECUSDT,EOSUSDT,year,week,day,week_hour
DT,,,,,,,,,,,,,,,,
2019-01-01 01:00:00,-0.00057,-0.68,-0.22,-0.00013,-0.0604,-0.00035,-10.62,-0.0261,-0.00109,NaN,NaN,-0.0132,2019,1,2,0
2019-01-01 02:00:00,0.00001,-0.42,-0.02,0.00002,-0.0470,-0.00009,0.31,-0.0243,-0.00168,NaN,NaN,0.0029,2019,1,2,1
2019-01-01 03:00:00,0.00120,0.60,0.06,0.00014,-0.0703,0.00029,3.13,0.0465,0.00234,NaN,NaN,0.0149,2019,1,2,2
2019-01-01 04:00:00,-0.00003,1.03,0.14,0.00013,0.0527,0.00023,-0.42,0.0126,0.00179,NaN,NaN,0.0004,2019,1,2,3
2019-01-01 05:00:00,0.00065,0.40,0.09,0.00011,0.0168,0.00011,7.23,0.0528,0.00081,NaN,NaN,0.0076,2019,1,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-31 19:00:00,0.00160,-0.33,0.40,0.00052,-0.8000,-0.00290,-57.20,0.1700,0.00120,0.6,0.1,0.0080,2022,22,2,43
2022-05-31 20:00:00,0.00210,19.45,0.60,0.00100,1.5000,0.00500,187.94,0.1600,0.00550,2.8,0.8,0.0090,2022,22,2,44
2022-05-31 21:00:00,-0.00090,-6.20,-0.20,-0.00023,-0.9000,0.00950,-93.64,-0.0300,0.00100,0.6,-0.2,-0.0010,2022,22,2,45


In [5]:
Hours = list(range(0, 168, 24))
Hours

[0, 24, 48, 72, 96, 120, 144]

In [6]:
def model(index: int = (0, len(tickers))):
    ticker = tickers[index]
    Profile = F.groupby('week_hour')[ticker].mean().cumsum()
    P_2019 = F['2019'].groupby('week_hour')[ticker].mean().cumsum()
    P_2020 = F['2020'].groupby('week_hour')[ticker].mean().cumsum()
    P_2021 = F['2021'].groupby('week_hour')[ticker].mean().cumsum()
    P_2022 = F['2022'].groupby('week_hour')[ticker].mean().cumsum()
    layout = dict(title={'text': f'{ticker}: {F.index.date.min().strftime("%b %Y")}..{F.index.date.max().strftime("%b %Y")}', 'x': 0.5})
    return 0, locals()

In [7]:
box = interactFigure(model, height=500,
    Profile=dict(width=8, opacity=0.4),
    P_2019=dict(width=2, opacity=0.3),
    P_2020=dict(width=2, opacity=0.3),
    P_2021=dict(width=2, opacity=0.3),
    P_2022=dict(width=3, opacity=0.3),
    Weekdays=dict(x=Hours, y=[0] * len(Hours), mode="text", textposition="top right", text=weekdays)
)

In [8]:
# add vertical daily lines
[box.children[1].add_vline(x=hour, line_color='gray', line_dash='dot') for hour in Hours + [168]];

In [9]:
box

    …